In [256]:
from bs4 import BeautifulSoup as bs
import urllib2
import pandas as pd
pd.set_option('display.max_columns', None)


csv_data = []

for page_number in range(1,481):
    print "Now analysing page: " , page_number
    target = 'http://www.trackitt.com/usa-immigration-trackers/i140/page/' + str(page_number)
    html = urllib2.urlopen(target)
    soup = bs(html,"html.parser")
    table = soup.find("table",{"id":"myTable01"})
    trs = table.find_all('tr')
    for tr in trs:
        tds = tr.find_all('td')
        temp = []
        for td in tds:
            inner_text = td.text
            strings = inner_text.split("\n")
            temp.extend(strings)
        csv_data.append(temp)
        
csv_data =[x for x in csv_data if len(x) == 30]
for sublist in csv_data:
    del sublist[4]
    del sublist[2]
#print csv_data[0], len(csv_data[0])

ths = trs[0].find_all('th')
table_header = []
for th in ths:
    th_content = th.text.lstrip().rstrip()
    strings = th_content.split("\n")
    table_header.extend(string for string in strings if string)

#print table_header,len(table_header)

Now analysing page:  1
Now analysing page:  2
Now analysing page:  3
Now analysing page:  4
Now analysing page:  5
Now analysing page:  6
Now analysing page:  7
Now analysing page:  8
Now analysing page: 

 9
Now analysing page:  10
Now analysing page:  11
Now analysing page:  12
Now analysing page:  13
Now analysing page:  14
Now analysing page:  15
Now analysing page:  16
Now analysing page:  17
Now analysing page:  18
Now analysing page:  19
Now analysing page:  20
Now analysing page:  21
Now analysing page:  22
Now analysing page:  23
Now analysing page:  24
Now analysing page:  25
Now analysing page:  26
Now analysing page:  27
Now analysing page:  28
Now analysing page:  29
Now analysing page:  30
Now analysing page:  31
Now analysing page:  32
Now analysing page:  33
Now analysing page:  34
Now analysing page:  35
Now analysing page:  36
Now analysing page:  37
Now analysing page:  38
Now analysing page:  39
Now analysing page:  40
Now analysing page:  41
Now analysing page:  42
Now analysing page:  43
Now analysing page:  44
Now analysing page:  45
Now analysing page:  46
Now analysing page:  47
Now analysing page:  48
Now analysing page:  49
Now analysing page:  50
Now analysing

In [280]:
df = pd.DataFrame(csv_data , columns = table_header)

In [261]:
df.to_csv('trackitt.csv', sep = ',', encoding = 'utf-8')  

In [493]:
import os
import csv
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

def get_path(filename):
    current_dir = os.getcwd()
    file_path = os.path.join(current_dir,filename)
    return file_path

def read_csv(filepath):
    data = []
    input_file = open(filepath,'rU')
    input_file_object = csv.reader(input_file)
    header = input_file_object.next()
    for row in input_file_object:
        data.append(row)
    data = np.array(data)
    return data
    
def read_as_pd(filepath):
    data_frame = pd.read_csv(filepath)
    return data_frame
    
input_file_path = get_path('trackitt.csv')
data = read_csv(input_file_path)      
df = read_as_pd(input_file_path)

In [494]:
#df.describe()

In [495]:
df = df.drop(['Unnamed: 0','Watch','Comments','More',
              'Priority Date','Most Recent LUD','Days Elapsed',
              'Notes','State','Last Updated','Username','Applicant Type',
              'Application Filed','USCIS Notice Date','USCIS Receipt Number',
              'RFE Received Date','Reason for RFE','RFE Replied Date','Case Added to Tracker'],1)

In [509]:
df.head()

,Nationality,Service Center,Category,USCIS Received Date,I-140/485 Filing,Processing Type,RFE Received?,Application Status,Approval/Denial Date,Total Processing Time
0,India,Texas,EB2,12/11/2014,non-concurrent,regular,NaN,approved,07/09/2015,216 days
1,India,Texas,EB2,06/23/2015,non-concurrent,premium,NaN,approved,07/02/2015,10 days
2,India,Texas,EB2,05/01/2015,concurrent,regular,NaN,pending,NaN,NaN
3,China,Texas,EB2-NIW,12/10/2014,non-concurrent,regular,NaN,approved,07/09/2015,225 days
4,India,Texas,EB2-NIW,09/30/2014,non-concurrent,regular,no,pending,NaN,NaN


In [609]:
#df["Nationality"].dropna()

In [624]:
df = df[df["Nationality"] != "Unknown"]
df = df[df["Nationality"] != "nan"]

In [625]:
len(df["Nationality"].unique())

156

In [690]:
for nation in df["Nationality"].unique():
        

India
China
Canada
South Korea
South Africa
Philippines
Sri Lanka
United States
United Kingdom
Nepal
Vietnam
Thailand
Bangladesh
Germany
Iran
Malaysia
Turkey
Taiwan
Egypt
Poland
UAE
Belarus
Sweden
Spain
Australia
Japan
Portugal
Trinidad & Tobago
Israel
Austria
Kazakhstan
Italy
Slovenia
Nigeria
Brazil
Cameroon
Russia
France
Pakistan
Colombia
Ireland
Singapore
Jamaica
Mexico
Kenya
Argentina
Belgium
Ukraine
Paraguay
Chile
Slovakia
Hong Kong
Oman
Venezuela
Jordan
Indonesia
Mongolia
Uganda
Romania
Lebanon
Bulgaria
Ghana
Sudan
Cyprus
New Zealand
Georgia
Congo (DR)
Saudi Arabia
Denmark
Serbia
Ethiopia
Peru
Syria
Qatar
Albania
nan
Zimbabwe
Uruguay
Norway
Myanmar
Hungary
Ecuador
Palestine
Finland
Dominica
Switzerland
Uzbekistan
Iceland
Barbados
Bolivia
Greece
Netherlands
Afghanistan
Bahamas
Moldova
Iraq
Saint Vincent
Guatemala
Burkina Faso
Bahrain
Botswana
Saint Lucia
Turkmenistan
Kuwait
Azerbaijan
Morocco
Mali
Estonia
Czech Republic
Croatia
Cape Verde
Lithuania
Algeria
Gambia
Liberia
Kyrgyzsta

In [639]:
data_array = df.reset_index().values
len(df)

22823

In [659]:
total_applicant = 0
pop = {}
for nation in df["Nationality"].unique():
    each_nation = data_array [0::,1] == nation
    total_applicant += len(data_array[each_nation,1])
    pop.update({nation : float(len(data_array[each_nation,1]))})

for key, value in pop.items(): 
    pop[key] = (value / total_applicant ) * 100.
#pop

In [660]:
import operator
sorted_pop = sorted(pop.items(), key=operator.itemgetter(1))
selected_nationality = dict(sorted_pop[-10:])
print "Population for different nationality :"
print "#######################################"
selected_nationality

Population for different nationality :
#######################################


{'Canada': 1.6637401229148376,
 'China': 6.308165057067604,
 'India': 65.27216856892011,
 'Iran': 1.079894644424934,
 'Nepal': 1.3432835820895521,
 'Pakistan': 2.0500438981562774,
 'Philippines': 1.8832309043020194,
 'South Korea': 2.5899912203687445,
 'Taiwan': 1.4749780509218613,
 'United Kingdom': 1.2071992976294996}

In [689]:
for nation in selected_nationality:
    EB2_for_each_nation = np.logical_and(data_array[0::,1] == nation , data_array[0::,3] == "EB2")
    EB2_NIW_for_each_nation = np.logical_and(data_array[0::,1] == nation , data_array[0::,3] == "EB2-NIW")
    print nation, len(data_array[EB2_for_each_nation]), len(data_array[EB2_NIW_for_each_nation])
    
    
#print EB2_for_each_nation

Canada 200 15
Pakistan 304 37
Iran 73 143
Philippines 203 14
Nepal 226 32
India 9288 374
United Kingdom 82 19
China 693 238
Taiwan 221 69
South Korea 280 136
